$$\operatorname{minimize}_{C, Q} t^{T} \operatorname{diag}(C) \\
\operatorname{subject to} \begin{array}{l} {\boldsymbol{C} \boldsymbol{X}-\boldsymbol{X}+\boldsymbol{Q}=0} \\ {\|\boldsymbol{Q}\|_{\infty, 1} \leq \tau} \\ {\boldsymbol{C} \geq 0} \\ {\operatorname{diag}(\boldsymbol{C}) \leq \mathbf{1}} \\ {\operatorname{Tr}(\boldsymbol{C})=r} \\ {C_{i j} \leq C_{j j} \text { for all } i, j}\end{array}$$


In [1]:
import numpy as np
from scipy.optimize import linprog

In [2]:
f = 40
r = 3
n = 100

In [3]:
F = np.random.random((f, r))
W = np.random.random((r, n))
X = F @ W

In [4]:
def build_a_eq(X=X, f=f, n=n):
    A = np.zeros((f*n + 1, f**2))
    for i in range(f):
        # Handle CX = X
        A[i*n:(i+1)*n, i*f:(i+1)*f] = X.T
        # Handle Tr(C) = r
        A[-1, i*(f+1)] = 1
    return A

In [5]:
def build_b_eq(X=X, r=r):
    b = np.zeros((f*n+1,))
    # Handle CX = X
    b[:-1] = X.flatten()
    # Handle Tr(C) = r
    b[-1] = r
    return b

In [6]:
def check_ab_eq(A, b, c, X=X, r=r):
    res_1 = (A @ c - b).sum()
    res_2 = (c.reshape(f, f) @ X - X).sum() + np.trace(c.reshape(f, f)) - r
    return  res_1 - res_2 < 1e-5

In [7]:
def build_a_ub(X=X, f=f, n=n):
    A = np.zeros((f**2, f**2))
    index_A = 0
    # Handle diag(C) <= 1
    for i in range(f):
        A[i, i*(f+1)] = 1
        index_A += 1
        
    # Handle C_ij <= C_jj  
    for j in range(f):
        for i in range(f):
            if i == j:
                continue
            C = np.zeros((f, f))
            C[j, j] = -1
            C[i, j] = 1
            A[index_A, :] = C.flatten()
            index_A += 1
    return A

In [8]:
def build_b_ub(f=f):
    b = np.zeros(f**2)
    b[:f] += 1
    return b

In [9]:
A_eq = build_a_eq()
b_eq = build_b_eq()

In [10]:
A_ub = build_a_ub()
b_ub = build_b_ub()

In [11]:
t = (np.random.random((f, f)) * np.eye(f)).flatten()

In [ ]:
opt = linprog(t, method='simplex', A_eq=A_eq, b_eq=b_eq, A_ub=A_ub, b_ub=b_ub)

/Users/antoine/Documents/master-datascience/Non-Negative-Matrix-Factorization/env/lib/python3.6/site-packages/scipy/optimize/_linprog_util.py:704: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  warn(redundancy_warning, OptimizeWarning)


In [ ]:
opt

In [ ]:
res = opt.x.reshape(f, f)

In [ ]:
res